In [1]:
!pip install react


In [2]:
!pip install network
!pip install plotly

In [6]:
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np
from datetime import datetime, timedelta

class RetailMediaNetwork:
    def __init__(self):
        # Enhanced sample data with more metrics
        self.retailers = pd.DataFrame([
            {"id": "R1", "name": "SuperMart", "size": 5000000, "revenue": 10000000,
             "ad_inventory": 1000000, "growth_rate": 0.15, "satisfaction": 0.92, "subset": 1},
            {"id": "R2", "name": "TechStore", "size": 3000000, "revenue": 7000000,
             "ad_inventory": 600000, "growth_rate": 0.12, "satisfaction": 0.88, "subset": 1},
            {"id": "R3", "name": "FreshMarket", "size": 2000000, "revenue": 5000000,
             "ad_inventory": 400000, "growth_rate": 0.18, "satisfaction": 0.95, "subset": 1}
        ])

        self.advertisers = pd.DataFrame([
            {"id": "A1", "name": "ElectroTech", "budget": 2000000, "roas": 3.5,
             "campaign_count": 15, "avg_ctr": 0.025, "category": "Electronics", "subset": 0},
            {"id": "A2", "name": "FoodCo", "budget": 1500000, "roas": 4.2,
             "campaign_count": 12, "avg_ctr": 0.032, "category": "Food", "subset": 0},
            {"id": "A3", "name": "BeautyBrand", "budget": 1000000, "roas": 3.8,
             "campaign_count": 8, "avg_ctr": 0.028, "category": "Beauty", "subset": 0}
        ])

        self.audiences = pd.DataFrame([
            {"id": "U1", "name": "Young Professionals", "size": 2500000,
             "engagement_rate": 0.12, "conversion_rate": 0.045, "avg_order": 85, "subset": 2},
            {"id": "U2", "name": "Families", "size": 3000000,
             "engagement_rate": 0.15, "conversion_rate": 0.038, "avg_order": 120, "subset": 2},
            {"id": "U3", "name": "Seniors", "size": 1500000,
             "engagement_rate": 0.08, "conversion_rate": 0.052, "avg_order": 95, "subset": 2}
        ])

        # Generate time series data for trends
        self.generate_historical_data()

        # Create network connections with weights
        self.connections = pd.DataFrame([
            # Advertisers to Retailers with spend amount
            ("A1", "R1", 800000), ("A1", "R2", 700000),
            ("A2", "R1", 600000), ("A2", "R3", 500000),
            ("A3", "R1", 400000), ("A3", "R2", 300000),
            # Retailers to Audiences with engagement strength
            ("R1", "U1", 0.8), ("R1", "U2", 0.7),
            ("R2", "U1", 0.75), ("R2", "U3", 0.85),
            ("R3", "U2", 0.9), ("R3", "U3", 0.7)
        ], columns=['source', 'target', 'weight'])

    def generate_historical_data(self):
        """Generate 12 months of historical data for trends"""
        # Fix date range generation
        end_date = datetime.now()
        dates = [end_date - timedelta(days=30*i) for i in range(12)][::-1]

        # Generate time series data for each metric
        self.historical_data = {
            'revenue': pd.DataFrame({
                'date': dates,
                'value': np.random.normal(1000000, 100000, 12) *
                        np.linspace(1, 1.5, 12)  # Upward trend
            }),
            'engagement': pd.DataFrame({
                'date': dates,
                'value': np.random.normal(0.1, 0.02, 12)
            }),
            'roas': pd.DataFrame({
                'date': dates,
                'value': np.random.normal(3.5, 0.3, 12)
            })
        }

    def create_network_graph(self, selected_node=None):
        """Create interactive network visualization"""
        G = nx.Graph()

        # Add nodes with all attributes including subset for layout
        for _, r in self.retailers.iterrows():
            G.add_node(r["id"], **r.to_dict())
        for _, a in self.advertisers.iterrows():
            G.add_node(a["id"], **a.to_dict())
        for _, u in self.audiences.iterrows():
            G.add_node(u["id"], **u.to_dict())

        # Add weighted edges
        for _, edge in self.connections.iterrows():
            G.add_edge(edge['source'], edge['target'], weight=edge['weight'])

        # Fixed multipartite layout
        pos = nx.multipartite_layout(G, subset='subset', scale=2)

        # Create edge traces
        edge_x = []
        edge_y = []
        edge_text = []

        for edge in G.edges(data=True):
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_text.append(f'Connection strength: {edge[2]["weight"]:,.2f}')

        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=1, color='#888'),
            hoverinfo='text',
            mode='lines',
            text=edge_text
        )

        # Create node traces
        node_traces = []
        colors = {'A': '#1f77b4', 'R': '#2ca02c', 'U': '#9467bd'}

        for node_type, color in colors.items():
            mask = [n.startswith(node_type) for n in G.nodes()]
            nodes = [n for n, m in zip(G.nodes(), mask) if m]

            if not nodes:
                continue

            node_x = [pos[node][0] for node in nodes]
            node_y = [pos[node][1] for node in nodes]

            node_text = []
            node_size = []

            for node in nodes:
                if node_type == 'A':
                    data = self.advertisers[self.advertisers['id'] == node].iloc[0]
                    text = (f"Advertiser: {data['name']}<br>"
                           f"Budget: ${data['budget']:,.2f}<br>"
                           f"ROAS: {data['roas']:.2f}<br>"
                           f"Campaigns: {data['campaign_count']}")
                    size = data['budget'] / 50000
                elif node_type == 'R':
                    data = self.retailers[self.retailers['id'] == node].iloc[0]
                    text = (f"Retailer: {data['name']}<br>"
                           f"Size: {data['size']:,}<br>"
                           f"Revenue: ${data['revenue']:,.2f}")
                    size = data['size'] / 50000
                else:
                    data = self.audiences[self.audiences['id'] == node].iloc[0]
                    text = (f"Audience: {data['name']}<br>"
                           f"Size: {data['size']:,}<br>"
                           f"Engagement: {data['engagement_rate']:.1%}")
                    size = data['size'] / 50000

                node_text.append(text)
                node_size.append(size)

            node_trace = go.Scatter(
                x=node_x, y=node_y,
                mode='markers',
                hoverinfo='text',
                text=node_text,
                marker=dict(
                    size=node_size,
                    color=color,
                    line_width=2,
                    line=dict(color='white')
                ),
                name={'A': 'Advertisers', 'R': 'Retailers', 'U': 'Audiences'}[node_type]
            )
            node_traces.append(node_trace)

        # Create figure
        fig = go.Figure(
            data=[edge_trace] + node_traces,
            layout=go.Layout(
                title='Retail Media Network Analysis',
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                plot_bgcolor='white'
            )
        )

        return fig

    def create_metrics_dashboard(self):
        """Create metrics visualization"""
        fig = go.Figure()

        # Revenue Trend
        fig.add_trace(go.Scatter(
            x=self.historical_data['revenue']['date'],
            y=self.historical_data['revenue']['value'],
            name='Revenue',
            line=dict(color='#1f77b4')
        ))

        # ROAS Trend
        fig.add_trace(go.Scatter(
            x=self.historical_data['roas']['date'],
            y=self.historical_data['roas']['value'],
            name='ROAS',
            yaxis='y2',
            line=dict(color='#2ca02c')
        ))

        fig.update_layout(
            title='Network Performance Metrics',
            xaxis=dict(title='Date'),
            yaxis=dict(title='Revenue ($)'),
            yaxis2=dict(title='ROAS', overlaying='y', side='right'),
            hovermode='x unified'
        )

        return fig

    def show_dashboard(self):
        """Display the complete dashboard"""
        # Create tabs
        tab_titles = ['Network View', 'Performance Metrics', 'Analysis']
        tabs = widgets.Tab()
        children = [widgets.Output() for _ in range(len(tab_titles))]
        tabs.children = children
        for i, title in enumerate(tab_titles):
            tabs.set_title(i, title)

        # Fill tabs with content
        with children[0]:
            network_fig = self.create_network_graph()
            display(network_fig)

        with children[1]:
            metrics_fig = self.create_metrics_dashboard()
            display(metrics_fig)

        with children[2]:
            total_revenue = self.retailers['revenue'].sum()
            avg_satisfaction = self.retailers['satisfaction'].mean()
            total_audience = self.audiences['size'].sum()

            html_content = f"""
            <div style='padding: 20px;'>
                <h3>Network Summary</h3>
                <p>Total Network Revenue: ${total_revenue:,.2f}</p>
                <p>Average Retailer Satisfaction: {avg_satisfaction:.1%}</p>
                <p>Total Audience Reach: {total_audience:,}</p>
            </div>
            """
            display(HTML(html_content))

        display(tabs)

# Function to initialize and display the dashboard
def show_retail_media_dashboard():
    network = RetailMediaNetwork()
    network.show_dashboard()

# Run the visualization
show_retail_media_dashboard()